# Some basics

> Fill in a module description here

In [ ]:
#| default_exp tutorials/CubeCoordinates

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev import nbdev_export

In [ ]:
nbdev_export()

In [ ]:
#| export
#| hide
#import nbdev; nbdev.nbdev_export()
import sys
import math
from fastcore.basics import patch

#| export
## Introduction

The purpose of this library is to generate hex maps that are used in board games.

#| export
## Getting Started

In [ ]:
#| export
from HexMagic.primitives import  MapCord 
from HexMagic.primitives import Hex


In [ ]:
#| export

sampleHex = Hex(radius=50,center=MapCord(100,100))

sampleHex.vertices(), "\n-- We round edges so we can compare --\n", sampleHex.edges()

([(143,75), (143,125), (100,150), (56,125), (56,75), (99,50)],
 '\n-- We round edges so we can compare --\n',
 {Edge(x1=56, y1=75, x2=56, y2=125),
  Edge(x1=56, y1=75, x2=99, y2=50),
  Edge(x1=56, y1=125, x2=100, y2=150),
  Edge(x1=99, y1=50, x2=143, y2=75),
  Edge(x1=100, y1=150, x2=143, y2=125),
  Edge(x1=143, y1=75, x2=143, y2=125)})

In [ ]:
#| export
from HexMagic.styles import StyleCSS,  SVGBuilder

In [ ]:
#| export
#some drawing setup 
hexStyle = StyleCSS("HexStyle",fill="yellow",stroke="blue",stroke_width=2)
canvas = SVGBuilder()
canvas.width=200 ;canvas.height=200
canvas.add_style(hexStyle)

#add our hex to the canvas
sampleHex = Hex(radius=50,center=MapCord(100,100),style=hexStyle)
canvas.adjust("main",sampleHex.svg())

#show our work
canvas.show()
canvas.adjust("main",sampleHex.svg())

#show our work
canvas.show()

#| export
Hex, short for hexagon, is a six-sided polygon that can be laid out in a honeycomb pattern that is space-efficient and strong. It also has a nicer perimeter than a square. This library groups them in a hexgrid, a honeycomb type structure

In [ ]:
from HexMagic.primitives import HexGrid, PrimitiveDemo

In [ ]:

sampleGrid = PrimitiveDemo().sampleGrid(fill="red")
sampleGrid.builder.show()

#| export
The `HexGrid` has a list of `Hex`, and we initialize them with a common style. The hexes are ordered starting from the top left.

In [ ]:
#| export

grid = PrimitiveDemo().sampleGrid(makeLabels=True)
grid.builder.show()

#| export
### Absolute versus Realtive positions

This integer base system is great for absolute postioning, but is problematic from a realtive one. If we look at hex 12 two of its neighbors (11,13) are off by one. But the rest are going to be tricky since we need the number of rows and columns. Also even rows are different than odd ones.

What we need are directions.







In [ ]:
#| export
from HexMagic.primitives import HexPosition, MapPath
grid = PrimitiveDemo().sampleGrid()

#mark the treasure
grid.hexes[grid.midpoint].label = "X"

#lets have a directions layer
arrowLayer = ""

#Use the realtive position class called HexPosition
for position in HexPosition.directions():
    i = grid.hexposition_to_index(position, 12)
    grid.hexes[i].label = position.label
    arrowLayer += grid.arrow(12,i)

grid.update()

#we can add a layer or update one in a builder using the adjust method
grid.builder.adjust("arrows",arrowLayer)

grid.builder.show()

In [ ]:
#| export
eastDir = HexPosition.E
eastDir,  2 * eastDir

(HexPosition(1, 0, -1, 'E'), HexPosition(2, 0, -2))

In [ ]:
#| export
def showEast(middleIndex=12):
    grid = PrimitiveDemo().sampleGrid()

    #lets have a directions layer
    arrowLayer = ""
    
    #Mark our starting point
    grid.hexes[middleIndex].label = str(middleIndex)

    #Get a Direction
    eastDir = HexPosition.E

    #go part way
    i = grid.hexposition_to_index(eastDir, middleIndex)
    grid.hexes[i].label = "near"
    arrowLayer += grid.arrow(middleIndex,i)

    #keep going
    farEast = 2 * eastDir 
    j = grid.hexposition_to_index(farEast, middleIndex)
    grid.hexes[j].label = "far"
    arrowLayer += grid.arrow(i,j)

    grid.update()
    grid.builder.adjust("arrows",arrowLayer)
    return grid.builder.show()
showEast(12)


#| export
The nice part is that there is nothing special about where we start. It doesn't have to be the center

In [ ]:
#| export
showEast(6)

#| export
A common journey that you might want to do is to go out a certain distance and walk around in a ring.

In [ ]:
#| export
def showRing(middleIndex=12,ring=1):
    grid = PrimitiveDemo().sampleGrid()
    
    grid.hexes[middleIndex].label = "X"


    #lets have a directions layer
    arrowLayer = ""
    #Get a Direction
    eastDir = HexPosition.E

    #go out to the edge
    s = middleIndex
    for k in range(ring):
        i = grid.hexposition_to_index(eastDir, s)
        grid.hexes[i].label = f"r {k+1}"
        arrowLayer += grid.arrow(s,i)
        s = i

    origin = HexPosition.origin()

    #now lets go through the positions. by default it starts in the east like the sunrise.
    positions = origin.ring(ring)
    for pos in positions:
        i = grid.hexposition_to_index(pos,middleIndex ) 
        grid.hexes[i].label = f"r {pos.label}"
        if s != i:
            arrowLayer += grid.arrow(s,i)
        s = i

    grid.update()
    grid.builder.adjust("arrows",arrowLayer)
    return grid.builder.show()
    
showRing(12)

In [ ]:
#| export
showRing(12,ring=2)

#| export
### Coordinates

When representing hex directions, we need a coordinate system with several key properties:

1. **Additive**: Directions should combine naturally (e.g., two "east" moves = moving two hexes east)
2. **Reversible**: Every direction has a clear opposite (West is the reverse of East)
3. **Computationally efficient**: Operations should be fast and avoid floating-point errors

Since hexagons exist in the x,y plane, let's consider our options:

#### Cartesian Coordinates
The familiar x,y system works well for horizontal movements, but diagonal vectors involve factors of √3, requiring floating-point arithmetic and introducing potential rounding errors.

#### Polar Coordinates
Angles are clean multiples of 60°, but constant conversion between polar and Cartesian for rendering makes this impractical.

#### Dive a bit deeper
We have three pairs of directions (E/W,NW/SE,NE/SW) so we are going to need at 2 bits to repesent them. Lets build them

1. East can be (1, 0)
2. This makes West (-1, 0)
3. Let's make Southeast (0, 1)
4. Which makes Northwest (0, -1)
5. To get to Northeast using what we have built already. we go east (1,0), and then north west (0, -1) for (1,-1)
6. And that makes Southwest the opposite of Northeast (-1,1)




In [ ]:
def axisGrid(hexDim = 2, fill = "white",makeLabels = False):

    #drawing set up
    hexStyle = StyleCSS("HexStyle",fill=fill,stroke="blue",stroke_width=2)
    labelStyle = StyleCSS("labelStyle",fill=fill,stroke="black",stroke_width=1)
    
    grid = HexGrid.centered(hexDim,radius=40,style=hexStyle)

    #Need to add styles as we go along
    grid.builder.add_style(labelStyle)

    #itterate through the hexes
    for i in range(len(grid.hexes)):
        pos = grid.index_to_hexposition(i)
        if makeLabels:
            grid.hexes[i].label = f"{pos.q}, {pos.r}" 
        grid.hexes[i].labelStyle = labelStyle.name

    grid.update()

    arrowLayer = ""
   
    for pos in [hexDim * x for x in [HexPosition.E,HexPosition.W,HexPosition.NE,HexPosition.SW]]:
        index = grid.hexposition_to_index(pos)
        arrowLayer += grid.arrow(grid.midpoint,index)

    grid.builder.adjust("axis",arrowLayer)
    return grid
    

grid = axisGrid(4,makeLabels=True)
grid.builder.show()

#| export
#### Cube Coordinates
What we have actually created are the first two digits of **cube coordinates** - a three-axis system (q, r, s) where:
- East: (1, 0, -1)
- West: (-1, 0, 1)
- Southeast: (0, 1, -1)
- Northwest: (0, -1, 1)
- Northeast: (1, -1, 0)
- Southwest: (-1, 1, 0)

The key constraint: **q + r + s = 0** always.

#### Why Cube Coordinates?

1. **Integer arithmetic only**: All operations use integers - no floating-point errors
2. **Natural addition**: Adding positions is simple component-wise addition
3. **Built-in validation**: The q + r + s = 0 constraint acts like double-entry accounting, catching errors immediately
4. **Distance calculation**: Manhattan distance in cube space equals hex distance: `(|q| + |r| + |s|) / 2`
5. **Rotation and reflection**: These operations become simple transformations of the three coordinates
6. **Line drawing**: Interpolating between hexes is straightforward with linear interpolation
7. **Range queries**: Finding all hexes within N steps is elegant: all positions where `(|q| + |r| + |s|) / 2 ≤ N`

The redundancy of the third coordinate isn't waste - it's a feature that makes many algorithms simpler and more robust.


**Flip** inverts all coordinates:
```
flip(q, r, s) = (-q, -r, -s)
```

**Geometric meaning**: Rotation by 180° around the origin, or equivalently, reflection through the origin.

**Example**:

In [ ]:
#| export
def flip(h):
    return (-HexPosition.direction(h)).desc()


flip("E")

'W'

#| export

**Geometric meaning**: Rotation by 180° around the origin, or equivalently, reflection through the origin.

**Example**:
```
flip(1, 0, -1) = (-1, 0, 1)
E → W (East becomes West)
```

**Key property**: Flip creates **opposite pairs**. Every direction has an opposite, and moving in direction D then flip(D) returns you to the start:
```
H + D + flip(D) = H + D - D = H
```

This is why addition and subtraction work naturally:
```
(2, 3, -5) - (1, 0, -1) = (2, 3, -5) + flip(1, 0, -1) 
                        = (2, 3, -5) + (-1, 0, 1)
                        = (1, 3, -4)
```

**Algebraic significance**: Flip gives us **additive inverses**, a requirement for forming a group (and later, a ring).

### 2. Roll (60° Rotation)

**Roll** cycles the coordinates by both shifting (3 states) and flipping (2 states):
```
roll(q, r, s) = (-s, -q, -r)
```

**Why the negations?** We need to preserve the constraint `q + r + s = 0`:
```
-s + (-q) + (-r) = -(s + q + r) = -0 = 0 ✓
```

In [ ]:
#| export

def roll(h):
    """ Geometric meaning**: Rotation by 60° counter-clockwise around the origin."""
    pos = -HexPosition.direction(h)
    return HexPosition(pos.s,pos.q,pos.r).desc()
   

#Rolling six times returns to the original position, giving us **cyclic order**.
start = "E"
for I in range(6):
    print(start)
    start = roll(start)

#**Algebraic significance**: Roll generates **rotational symmetry**, the six-fold symmetry that makes hexagons special.


E
NE
NW
W
SW
SE


#| export
Notice that `roll³(E) = flip(E)`, so roll alone could generate everything. But having **both** operations makes the structure clearer:
- **Roll** gives us rotation around the circle
- **Flip** gives us opposite pairs

### The Group Structure

The six directions form a **cyclic group** of order 6 under composition, isomorphic to **ℤ/6ℤ** (integers modulo 6).

But here's the magic: **positions** (not just directions) form something richer—a structure where:
1. We can **add** positions: `(q₁, r₁, s₁) + (q₂, r₂, s₂) = (q₁+q₂, r₁+r₂, s₁+s₂)`
2. We can **scale** by integers: `n · (q, r, s) = (nq, nr, ns)`
3. We have **additive inverses** (flip): `-(q, r, s) = (-q, -r, -s)`

This makes hex positions a **ℤ-module** (like a vector space, but over integers instead of real numbers).

## Why Three Coordinates? The Algebraic Answer

Here's the deep reason: **cube coordinates embed the 2D hexagonal lattice into 3D space in a way that preserves all symmetries**.

### Option 1: Two Coordinates (Axial)

We could use `(q, r)` and compute `s = -q - r`. This works, but:
- Addition is still natural: `(q₁, r₁) + (q₂, r₂) = (q₁+q₂, r₁+r₂)`
- But **rotation becomes asymmetric**:
  ```
  roll(q, r) = (q + r, -q)   [derived from roll(q, r, -q-r)]
  ```
  This formula isn't obviously a rotation! The symmetry is hidden.
Notice that `roll³(E) = flip(E)`, so roll alone could generate everything. But having **both** operations makes the structure clearer:
- **Roll** gives us rotation around the circle
- **Flip** gives us opposite pairs



#| export
### Option 2: Three Coordinates (Cube)

With `(q, r, s)`:
- Addition: `(q₁, r₁, s₁) + (q₂, r₂, s₂) = (q₁+q₂, r₁+r₂, s₁+s₂)` ✓
- Flip: `(-q, -r, -s)` (negate all) ✓  
- Roll: `(-s, -q, -r)` (permute and negate) ✓

All three operations treat the coordinates **symmetrically**. The three-fold symmetry of the constraint `q + r + s = 0` matches the six-fold symmetry of the hexagon (6 = 2 × 3).

### The Lie Algebra Connection

For those familiar with advanced mathematics: the constraint `q + r + s = 0` defines a **2D subspace** of ℤ³ that's **closed under rotation**. This is the **root lattice** of type A₂ (or equivalently, the weight lattice of SU(3)).

The roll operation is a **discrete rotation** preserving this lattice. Having all three coordinates lets us express this rotation as a simple permutation matrix, revealing the underlying symmetry group.

## Ring Structure: Combining Operations

Cube coordinates form an **abelian group** under addition:
- **Closure**: Sum of two hex coords is a hex coord (constraint preserved)
- **Associativity**: `(A + B) + C = A + (B + C)`
- **Identity**: `(0, 0, 0)` is the origin
- **Inverses**: flip gives `-A` for every `A`
- **Commutativity**: `A + B = B + A`

We can also define **scalar multiplication** by integers:
```
n · (q, r, s) = (nq, nr, ns)
```

This makes cube coordinates a **ℤ-module**, which is the algebraic structure that captures:
- Linear combinations of positions
- Scaling by integer factors
- All while preserving the hex lattice


#| export
## Algorithims

#| export
### 1. Distance Calculation

The **distance** between two hexes is:
```python
@patch
def distance(self: HexPosition, other: HexPosition) -> int:
    """Cube coordinate distance using numpy"""
    return int(np.abs(self._coords - other._coords).sum() // 2)
```

With three coordinates, this formula is **symmetric**. With axial `(q, r)`, you'd need:
```python
def distance_axial(h1, h2):
    dq = h1.q - h2.q
    dr = h1.r - h2.r
    return max(abs(dq), abs(dr), abs(dq + dr))
```
Less intuitive and asymmetric.

### 2. Rotation

Rotating a hex coordinate by 60° counter-clockwise:
```python
@patch
def rotate_left(self: HexPosition) -> 'HexPosition':
    """Rotate 60° counter-clockwise: (q,r,s) -> (-s,-q,-r)"""
    return HexPosition(*(-np.roll(self._coords, 1)))
```

Rotating by 60° clockwise:
```python
@patch
def rotate_right(self: HexPosition) -> 'HexPosition':
    """Rotate 60° clockwise: (q,r,s) -> (-r,-s,-q)"""
    return HexPosition(*(-np.roll(self._coords, -1)))
```

Both are **simple permutations**. With two coordinates, rotation formulas are more complex.


#| export
To draw a line between hexes, we use **linear interpolation** in cube coordinates:
```python
@patch
def line_to(self: HexPosition, other: HexPosition) -> list[HexPosition]:
    """Get line of hexes from self to other using linear interpolation"""
    n = self.distance(other)
    if n == 0:
        return [self]

    results = []
    for i in range(n + 1):
        t = i / n
        # Lerp in cube coordinates using numpy
        coords = self._coords + (other._coords - self._coords) * t
        results.append(HexPosition._cube_round(*coords))

    return results
```

In [ ]:
#| export

def hexes_in_range(n):
    """ finding all hexes within distance N from origin: """
    results = []
    for q in range(-n, n + 1):
        for r in range(max(-n, -q - n), min(n, -q + n) + 1):
            s = -q - r
            results.append(HexPosition(q, r, s))
    return results

hexes_in_range(3)

[HexPosition(-3, 0, 3),
 HexPosition(-3, 1, 2),
 HexPosition(-3, 2, 1),
 HexPosition(-3, 3, 0),
 HexPosition(-2, -1, 3),
 HexPosition(-2, 0, 2),
 HexPosition(-2, 1, 1),
 HexPosition(-2, 2, 0),
 HexPosition(-2, 3, -1),
 HexPosition(-1, -2, 3),
 HexPosition(-1, -1, 2),
 HexPosition(-1, 0, 1),
 HexPosition(-1, 1, 0),
 HexPosition(-1, 2, -1),
 HexPosition(-1, 3, -2),
 HexPosition(0, -3, 3),
 HexPosition(0, -2, 2),
 HexPosition(0, -1, 1),
 HexPosition(0, 0, 0),
 HexPosition(0, 1, -1),
 HexPosition(0, 2, -2),
 HexPosition(0, 3, -3),
 HexPosition(1, -3, 2),
 HexPosition(1, -2, 1),
 HexPosition(1, -1, 0),
 HexPosition(1, 0, -1),
 HexPosition(1, 1, -2),
 HexPosition(1, 2, -3),
 HexPosition(2, -3, 1),
 HexPosition(2, -2, 0),
 HexPosition(2, -1, -1),
 HexPosition(2, 0, -2),
 HexPosition(2, 1, -3),
 HexPosition(3, -3, 0),
 HexPosition(3, -2, -1),
 HexPosition(3, -1, -2),
 HexPosition(3, 0, -3)]

#| export
### Disscussion

#### Advanced Topic: Why Roll Uses Negation

A common question: why is roll `(-s, -q, -r)` instead of `(s, q, r)`?

**Answer**: We need `q + r + s = 0` to remain true after rolling.

If we tried `roll'(q, r, s) = (s, q, r)`:
```
s + q + r = -(q + r + s) = -0 = 0  (wait, that works!)
```

Actually, `(s, q, r)` does preserve the constraint! But:
```
roll'(1, 0, -1) = (-1, 1, 0)
```

This is **clockwise** rotation, not counter-clockwise. The conventional choice is:
- **Counter-clockwise**: `(-s, -q, -r)`
- **Clockwise**: `(-r, -s, -q)`

Both preserve the constraint. We choose counter-clockwise to match standard mathematical convention (positive angles rotate counter-clockwise).

The negations ensure the **direction** of rotation aligns with our coordinate system.

#### Comparison to Modular Arithmetic

You mentioned the **modular operator** as a more common way to create cyclic structures. Let's compare:

##### Modular Arithmetic (ℤ/6ℤ)

In **ℤ mod 6**, we have six elements: {0, 1, 2, 3, 4, 5}

Operations:
- **Addition**: `(a + b) mod 6`
- **Negation**: `(-a) mod 6 = (6 - a) mod 6`
- **Multiplication**: `(a · b) mod 6`

Example: Directions as integers
```
E = 0, NE = 1, NW = 2, W = 3, SW = 4, SE = 5
```

To rotate by 60°: `new_dir = (old_dir + 1) mod 6`  
To flip (180°): `new_dir = (old_dir + 3) mod 6`

**Advantages**:
- Single number to store per direction
- Fast operations with bit masking (`& 0x3F` for mod 64)

**Disadvantages**:
- **Positions** can't be represented this way! `(0 + 2) mod 6 = 2`, but the position "2 east" is different from "1 east + 1 northeast"
- No natural **distance metric**
- No **interpolation** or continuous paths

##### Cube Coordinates

Positions: `(q, r, s)` with `q + r + s = 0`

Operations:
- **Addition**: Component-wise
- **Negation**: Component-wise
- **Rotation**: Permutation with sign

**Advantages**:
- Positions AND directions in same framework
- Natural distance, interpolation, range queries
- Geometric intuition (points in 3D space)

**Disadvantages**:
- 3× memory vs single direction index
- Requires maintaining constraint


In [ ]:
#| export
def line_demo(start_idx=20, end_idx=8):
    """Demo showing line drawing between two hexes using cube coordinates"""
    grid = PrimitiveDemo().sampleGrid()
    # Clear the axis arrows from sampleGrid
    grid.builder.adjust("axis", "")
    
    # Mark start and end
    grid.hexes[start_idx].label = "Start"
    grid.hexes[end_idx].label = "End"

    location = grid.index_to_hexposition
    
    # Get the hex positions. This defaults to postion 0
    start_pos = location(start_idx)
    end_pos = location(end_idx)
    
    # Draw a line between them using cube coordinate interpolation
    path_positions = start_pos.line_to(end_pos)
    
    arrowLayer = ""
    prev_idx = start_idx
    
    for i, pos in enumerate(path_positions[1:], 1):  # Skip first (it's the start)
        curr_idx = grid.hexposition_to_index(pos)
        
        # Label intermediate hexes
        if curr_idx != end_idx and curr_idx >= 0:
            grid.hexes[curr_idx].label = str(i)
        
        # Draw arrow if valid index
        if curr_idx >= 0:
            arrowLayer += grid.arrow(prev_idx, curr_idx)
            prev_idx = curr_idx
    
    grid.update()
    grid.builder.adjust("arrows", arrowLayer)
    return grid.builder.show()

line_demo(20, 8)


In [ ]:
#| export
def range_demo( max_distance=2):
    """Demo showing all hexes within a certain distance"""
    
    grid = PrimitiveDemo().sampleGrid(3, fill="lightgray")
    center_idx = grid.midpoint

    # Mark center
    grid.hexes[center_idx].label = "0"
    
    # Get center position
    center_pos = grid.index_to_hexposition(center_idx)
    
    # Color hexes by distance
    colors = [StyleCSS(x,fill=x,stroke="blue",stroke_width=2) for x in ["green","yellow", "orange", "red", "purple"]]
    for color in colors:
        grid.builder.add_style(color)
    
    for distance in range(1, max_distance + 1):
        # Get all positions at this distance
        positions = center_pos.ring(distance)
        
        for pos in positions:
            try:
                idx = grid.hexposition_to_index(pos, center_idx)
                if 0 <= idx < len(grid.hexes):
                    grid.hexes[idx].label = str(distance)
                    grid.hexes[idx].style = colors[distance-1]
                    # You could change fill color here if we modify the hex style
            except:
                pass  # Position outside grid
    
    grid.update()
    return grid.builder.show()

range_demo(3)


In [ ]:
#| export
def rotation_demo(center_idx=12):
    """Demo showing rotation around a center hex"""
    grid = PrimitiveDemo().sampleGrid()
    # Clear the axis arrows from sampleGrid
    grid.builder.adjust("axis", "")
    
    grid.hexes[center_idx].label = "Center"
    
    # Start with a position to the east
    start_pos = HexPosition.E
    start_idx = grid.hexposition_to_index(2 * start_pos , center_idx)
    grid.hexes[start_idx].label = "0°"
    
    arrowLayer = ""
    prev_idx = start_idx
    
    # Rotate around 60° at a time
    for i in range(1, 6):
        # Rotate the position
        rotated_pos = 2 * start_pos.rotate(-i)
        curr_idx = grid.hexposition_to_index(rotated_pos, center_idx)
        
        grid.hexes[curr_idx].label = f"{i*60}°"
        arrowLayer += grid.arrow(prev_idx, curr_idx)
        prev_idx = curr_idx
    
    # Close the loop
    arrowLayer += grid.arrow(prev_idx, start_idx)
    
    grid.update()
    grid.builder.adjust("arrows", arrowLayer)
    return grid.builder.show()

rotation_demo(12)


In [ ]:
#| export
def demo_region_spiral_with_arrows():
    """Demo showing a region growing in a spiral pattern with arrows"""
    grid = PrimitiveDemo().sampleGrid(4, fill="lightgray")
    
    # Create gradient of colors
    colors = StyleCSS.seaborn("BuGn",37)
    
    arrow_style = StyleCSS("spiral_arrow", stroke="#333333", stroke_width=2, fill="none")
    
    for color in colors:
        grid.builder.add_style(color)
    grid.builder.add_style(arrow_style)
    
    center = grid.midpoint
    
    # Use the spiral method from HexPosition
    spiral_positions = HexPosition.origin().spiral(3)
    
    arrow_layer = ""
    prev_idx = None
    
    for i, pos in enumerate(spiral_positions):
        idx = grid.hexposition_to_index(pos, center)
        if 0 <= idx < len(grid.hexes):
            grid.hexes[idx].style = colors[i % len(colors)]
            grid.hexes[idx].label = str(i)
            
            # Draw arrow from previous hex to current
            if prev_idx is not None and prev_idx >= 0:
                arrow_layer += grid.arrow(prev_idx, idx, style=arrow_style)
            
            prev_idx = idx
    
    grid.update()
    grid.builder.adjust("arrows", arrow_layer)
    return grid.builder.show()



In [ ]:
#| export
demo_region_spiral_with_arrows()

### System of Patterns

Cube coordinates are meant for localized instruction, as in our head east. There are systems that are repetive instructions called Gosper curves that are very easy to make with the realtive nature of cube coordinates.

In [ ]:
class GosperCurve:
    """Generate Gosper curves using L-system rules and cube coordinates."""
    
    def __init__(self, order: int = 3):
        self.order = order
        
    @staticmethod
    def _expand_lsystem(axiom: str, rules: dict, iterations: int) -> str:
        """Expand an L-system string."""
        current = axiom
        for _ in range(iterations):
            current = ''.join(rules.get(c, c) for c in current)
        return current
    
    @classmethod
    def generate_path(cls, order: int, start: HexPosition = None) -> list[HexPosition]:
        """
        Generate a Gosper curve path using L-system.
        
        The Gosper curve L-system:
        - Axiom: A
        - Rules: A -> A-B--B+A++AA+B-
                 B -> +A-BB--B-A++A+B
        - +: turn left 60° (counter-clockwise)
        - -: turn right 60° (clockwise)
        """
        start = start or HexPosition.origin()
        
        expanded = cls._expand_lsystem(
            axiom="A",
            rules={'A': "A-B--B+A++AA+B-", 'B': "+A-BB--B-A++A+B"},
            iterations=order
        )
        
        path = [start]
        current_pos = start
        current_dir = HexPosition.direction("E")
        
        for symbol in expanded:
            if symbol in ('A', 'B'):
                current_pos = current_pos + current_dir
                path.append(current_pos)
            elif symbol == '+':
                current_dir = current_dir.rotate(1)
            elif symbol == '-':
                current_dir = current_dir.rotate(-1)
        
        return path
    
    @classmethod
    def generate_terdragon(cls, order: int, start: HexPosition = None) -> list[HexPosition]:
        """
        Generate a Terdragon curve (triangular variant).
        
        L-system:
        - Axiom: F
        - Rules: F -> F+F-F
        - +: turn left 120° (2 hex steps)
        - -: turn right 120°
        """
        start = start or HexPosition.origin()
        
        expanded = cls._expand_lsystem(
            axiom="F",
            rules={'F': "F+F-F"},
            iterations=order
        )
        
        path = [start]
        current_pos = start
        current_dir = HexPosition.direction("E")
        
        for symbol in expanded:
            if symbol == 'F':
                current_pos = current_pos + current_dir
                path.append(current_pos)
            elif symbol == '+':
                current_dir = current_dir.rotate(2)
            elif symbol == '-':
                current_dir = current_dir.rotate(-2)
        
        return path
    
    @classmethod
    def generate_peano_hex(cls, order: int, start: HexPosition = None) -> list[HexPosition]:
        """
        Generate a hexagonal Peano curve (space-filling).
        
        L-system:
        - Axiom: XF
        - Rules: X -> XF+YF+XF-F-YF-F-XF+YF-X
                 Y -> YF-XF-YF+F+XF+F+YF-XF+Y
        """
        start = start or HexPosition.origin()
        
        expanded = cls._expand_lsystem(
            axiom="XF",
            rules={
                'X': "XF+YF+XF-F-YF-F-XF+YF-X",
                'Y': "YF-XF-YF+F+XF+F+YF-XF+Y"
            },
            iterations=order
        )
        
        path = [start]
        current_pos = start
        current_dir = HexPosition.direction("E")
        
        for symbol in expanded:
            if symbol == 'F':
                current_pos = current_pos + current_dir
                path.append(current_pos)
            elif symbol == '+':
                current_dir = current_dir.rotate(1)   # 60° left, not 120°
            elif symbol == '-':
                current_dir = current_dir.rotate(-1)  # 60° right, not 120°

        
        return path
    
    @staticmethod
    def calculate_bounds(hex_path: list[HexPosition]) -> tuple[int, int, int, int]:
        """Calculate bounding box: (min_q, max_q, min_r, max_r)"""
        if not hex_path:
            return (0, 0, 0, 0)
        
        qs = [h.q for h in hex_path]
        rs = [h.r for h in hex_path]
        return (min(qs), max(qs), min(rs), max(rs))
    
    @staticmethod
    def center_path(hex_path: list[HexPosition]) -> list[HexPosition]:
        """Center a path around the origin."""
        if not hex_path:
            return []
        
        min_q, max_q, min_r, max_r = GosperCurve.calculate_bounds(hex_path)
        center_q = (min_q + max_q) // 2
        center_r = (min_r + max_r) // 2
        offset = HexPosition(center_q, center_r, -center_q - center_r)
        
        return [pos - offset for pos in hex_path]
    
    @staticmethod
    def path_to_mappath(hex_path: list[HexPosition], 
                        hex_radius: float = 20.0,
                        center: MapCord = None,
                        style: StyleCSS = None) -> MapPath:
        """Convert a HexPosition path to a MapPath for rendering."""
        center = center or MapCord(400, 400)
        style = style or StyleCSS("gosper", stroke="#2196F3", stroke_width=2, fill="none")
        
        pixel_points = [hexpos.pixel(hex_radius, center) for hexpos in hex_path]
        return MapPath(pixel_points, style)

In [ ]:
def demo_terdragon():
    """Demo showing Terdragon curve - a triangular fractal"""
    grid = PrimitiveDemo().sampleGrid(5, fill="#f0f0f0")
    
    path = GosperCurve.generate_terdragon(order=2)
    path = GosperCurve.center_path(path)
    
    print(f"Terdragon order 4: {len(path)} hexes")
    
    # Rainbow gradient
    colors = StyleCSS.seaborn("Spectral", len(path))
    for color in colors:
        grid.builder.add_style(color)
    
    arrow_style = StyleCSS("dragon_arrow", stroke="#333", stroke_width=1.5, fill="none")
    grid.builder.add_style(arrow_style)
    
    center = grid.midpoint
    arrow_layer = ""
    prev_idx = None
    
    for i, pos in enumerate(path):
        idx = grid.hexposition_to_index(pos, center)
        if 0 <= idx < len(grid.hexes):
            grid.hexes[idx].style = colors[i % len(colors)]
            
            if prev_idx is not None and prev_idx >= 0:
                arrow_layer += grid.arrow(prev_idx, idx, style=arrow_style)
            prev_idx = idx
    
    grid.update()
    grid.builder.adjust("arrows", arrow_layer)
    return grid.builder.show()

demo_terdragon()


Terdragon order 4: 10 hexes


In [ ]:
def demo_gosper_curves():
    """Demo showing Gosper curve with colored path on hex grid"""
    grid = PrimitiveDemo().sampleGrid(6, fill="white")
    
    # Generate a Gosper curve of order 2
    path = GosperCurve.generate_path(order=2)
    path = GosperCurve.center_path(path)
    
    print(f"Gosper curve order 2: {len(path)} hexes")
    
    # Create gradient colors for the path
    colors = StyleCSS.seaborn("viridis", len(path))
    for color in colors:
        grid.builder.add_style(color)
    
    arrow_style = StyleCSS("gosper_arrow", stroke="#FF5722", stroke_width=2, fill="none")
    grid.builder.add_style(arrow_style)
    
    center = grid.midpoint
    arrow_layer = ""
    prev_idx = None
    
    for i, pos in enumerate(path):
        idx = grid.hexposition_to_index(pos, center)
        if 0 <= idx < len(grid.hexes):
            grid.hexes[idx].style = colors[i % len(colors)]
            grid.hexes[idx].label = str(i) if i % 5 == 0 else ""
            
            if prev_idx is not None and prev_idx >= 0:
                arrow_layer += grid.arrow(prev_idx, idx, style=arrow_style)
            prev_idx = idx
    
    grid.update()
    grid.builder.adjust("arrows", arrow_layer)
    return grid.builder.show()

demo_gosper_curves()


Gosper curve order 2: 50 hexes


Any other ideas for cube coordinates?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Here are some more ideas for exploring cube coordinates:

**Algorithms & Patterns**
- **A* Pathfinding** - Cube coords make the heuristic trivial: Manhattan distance is just `(|Δq| + |Δr| + |Δs|) / 2`
- **Field of View** - Ray casting from a center hex, useful for games with visibility
- **Flood Fill** - Region growing with constraints (e.g., "fill all plains hexes connected to this one")
- **Hex Shadows** - Given obstacles, calculate which hexes are hidden from a light source

**Geometric Operations**
- **Reflection** - Reflect positions across any of the three axes (swap two coords and negate)
- **Wedges/Cones** - Select hexes in a 60° or 120° arc from a direction
- **Hex Intersection** - Find hexes that two rings/regions have in common

**Advanced Grids**
- **Toroidal Wrapping** - Wrap edges for games like Civilization
- **Irregular Borders** - Define regions by vertex lists, test hex containment
- **Multi-scale** - Hierarchical hexes (each hex contains 7 smaller hexes)

**Game Mechanics**
- **Movement with Costs** - Different terrain types have different movement costs
- **Zone of Control** - Mark hexes adjacent to enemy units
- **Hex-based Cellular Automata** - Conway's Game of Life on a hex grid (6 neighbors instead of 8)

**Visualization**
- **Heat maps** - Color hexes by distance, elevation, or any scalar field
- **Flow fields** - Show movement direction at each hex
- **Voronoi-like regions** - Assign hexes to nearest "seed" points

Would you like me to implement any of these? The pathfinding or field-of-view would pair nicely with your existing visualization setup.